In [9]:
import os
from pypdf import PdfReader

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Other Samples"

def extract_pdf_text_preserve_layout(pdf_path):
    reader = PdfReader(pdf_path)
    pages_text = {}
    for page_num, page in enumerate(reader.pages, start=1):
        page_text = page.extract_text(
            extraction_mode="layout",
            keep_blank_chars=True,
        )
        if page_text:
            pages_text[page_num] = page_text
    return pages_text

# Loop through PDFs
for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        pages_text = extract_pdf_text_preserve_layout(pdf_path)

        print(f"\n=== File: {filename} ===")
        for page_num, text in pages_text.items():
            print(f"\n--- Page {page_num} ---")
            print(text)



=== File: PO_CGR-05B40767 Longsleeve, T-Shirts_241219 - Copy.pdf ===

--- Page 1 ---
                                                                                                                   *05B00040767*
                                                                                                                 Order  05B00040767

Buy-from Vendor No.           9900960321                                           Order Date                    12-19-2024 (MM-DD-YYYY)
MANAMI FASHIONS LTD.                                                               Purchaser                     Catharina Grams
KABIRPUR, ASHULIA,                                                                 E-Mail                        catharina.grams@brands-
SAVAR,                                                                                                           fashion.com
Dhaka, 1349                                                                        Phone No.                     +49 (4181) 

In [19]:
import os
import re
from pypdf import PdfReader

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Other Samples"

def extract_pdf_text_preserve_layout(pdf_path):
    reader = PdfReader(pdf_path)
    pages_text = {}
    for page_num, page in enumerate(reader.pages, start=1):
        page_text = page.extract_text(
            extraction_mode="layout",
            keep_blank_chars=True,
        )
        if page_text:
            pages_text[page_num] = page_text
    return pages_text

def tokens_with_cols(line):
    """Return tokens with their starting column positions"""
    return [(m.group(), m.start()) for m in re.finditer(r"\S+", line)]

def extract_size_qty_mappings(text):
    """Extracts size-qty tables from text block"""
    pattern = r"(Size[^\n]+)\n\s*(Qty[^\n]+)"
    matches_iter = re.finditer(pattern, text)
    mappings = []

    for block_idx, m in enumerate(matches_iter, 1):
        size_line = m.group(1)
        qty_line  = m.group(2)

        size_cols = [(tok, col) for tok, col in tokens_with_cols(size_line) if tok.lower() != "size"]
        qty_cols  = [(tok, col) for tok, col in tokens_with_cols(qty_line) if tok.lower() != "qty"]

        mapping = {s: 0 for s, _ in size_cols}

        size_tokens = [s for s, _ in size_cols]
        qty_tokens  = [q for q, _ in qty_cols]

        # Case 1: counts match
        if len(size_tokens) == len(qty_tokens):
            for s_tok, q_tok in zip(size_tokens, qty_tokens):
                norm = q_tok.replace('.', '').replace(',', '')
                mapping[s_tok] = int(norm) if norm.isdigit() else 0
        else:
            # Case 2: align by column position
            size_ranges = []
            for i, (s_tok, s_col) in enumerate(size_cols):
                if i == 0:
                    s_start = -9999
                else:
                    prev_col = size_cols[i - 1][1]
                    s_start = (prev_col + s_col) // 2

                if i + 1 < len(size_cols):
                    next_col = size_cols[i + 1][1]
                    s_end = (s_col + next_col) // 2
                else:
                    s_end = 9999

                size_ranges.append((s_tok, s_start, s_end))

            for tok, q_col in qty_cols:
                norm = tok.replace('.', '').replace(',', '')
                if norm.isdigit():
                    q_val = int(norm)
                    for s_tok, s_start, s_end in size_ranges:
                        if s_start <= q_col < s_end:
                            mapping[s_tok] += q_val
                            break

        mappings.append(mapping)

    return mappings

# Loop through PDFs and extract tables
all_results = {}

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        pages_text = extract_pdf_text_preserve_layout(pdf_path)

        file_results = []
        for page_num, text in pages_text.items():
            mappings = extract_size_qty_mappings(text)
            if mappings:
                file_results.extend(mappings)

        if file_results:
            all_results[filename] = file_results

# --- Print Summary ---
for fname, blocks in all_results.items():
    print(f"\n=== Results from {fname} ===")
    for i, mapping in enumerate(blocks, 1):
        print(f"Block {i}: {mapping}")



=== Results from PO_CGR-05B40767 Longsleeve, T-Shirts_241219 - Copy.pdf ===
Block 1: {'XS': 400, 'S': 1320, 'M': 1220, 'L': 840, 'XL': 1280, 'XXL': 240, '3XL': 50, '4XL': 250, '5XL': 20}
Block 2: {'XS': 0, 'S': 250, 'M': 850, 'L': 90, 'XL': 430, 'XXL': 90, '3XL': 20, '4XL': 10}
Block 3: {'XXS': 600, 'XS': 4730, 'S': 14472, 'M': 22882, 'L': 12690, 'XL': 11410, 'XXL': 4100, '3XL': 800, '4XL': 1050, '5XL': 300, '6XL': 20, '7XL': 100}
Block 4: {'XXS': 0, 'XS': 1000, 'S': 4000, 'M': 8500, 'L': 400, 'XL': 5000, 'XXL': 2200, '3XL': 200, '4XL': 400, '5XL': 150, '6XL': 0, '7XL': 0}
Block 5: {'XXS': 0, 'XS': 7320, 'S': 15762, 'M': 25032, 'L': 15520, 'XL': 8350, 'XXL': 1100, '3XL': 1500, '4XL': 1450, '5XL': 150, '6XL': 0, '7XL': 120}
Block 6: {'XXS': 0, 'XS': 200, 'S': 252, 'M': 3002, 'L': 3500, 'XL': 300, 'XXL': 400, '3XL': 100, '4XL': 0, '5XL': 30, '6XL': 0, '7XL': 0}


In [22]:
import os
import re
from pypdf import PdfReader

pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Other Samples"

def extract_pdf_text_preserve_layout(pdf_path):
    reader = PdfReader(pdf_path)
    pages_text = {}
    for page_num, page in enumerate(reader.pages, start=1):
        page_text = page.extract_text(
            extraction_mode="layout",
            keep_blank_chars=True,
        )
        if page_text:
            pages_text[page_num] = page_text
    return pages_text

def tokens_with_cols(line):
    """Return tokens with their starting column positions"""
    return [(m.group(), m.start()) for m in re.finditer(r"\S+", line)]

def extract_size_qty_mappings(text):
    """Extracts size-qty tables from text block"""
    pattern = r"(Size[^\n]+)\n\s*(Qty[^\n]+)"
    matches_iter = re.finditer(pattern, text)
    mappings = []

    for block_idx, m in enumerate(matches_iter, 1):
        size_line = m.group(1)
        qty_line  = m.group(2)

        size_cols = [(tok, col) for tok, col in tokens_with_cols(size_line) if tok.lower() != "size"]
        qty_cols  = [(tok, col) for tok, col in tokens_with_cols(qty_line) if tok.lower() != "qty"]

        mapping = {s: 0 for s, _ in size_cols}

        size_tokens = [s for s, _ in size_cols]
        qty_tokens  = [q for q, _ in qty_cols]

        # Case 1: counts match
        if len(size_tokens) == len(qty_tokens):
            for s_tok, q_tok in zip(size_tokens, qty_tokens):
                norm = q_tok.replace('.', '').replace(',', '')
                mapping[s_tok] = int(norm) if norm.isdigit() else 0
        else:
            # Case 2: align by column position
            size_ranges = []
            for i, (s_tok, s_col) in enumerate(size_cols):
                if i == 0:
                    s_start = -9999
                else:
                    prev_col = size_cols[i - 1][1]
                    s_start = (prev_col + s_col) // 2

                if i + 1 < len(size_cols):
                    next_col = size_cols[i + 1][1]
                    s_end = (s_col + next_col) // 2
                else:
                    s_end = 9999

                size_ranges.append((s_tok, s_start, s_end))

            for tok, q_col in qty_cols:
                norm = tok.replace('.', '').replace(',', '')
                if norm.isdigit():
                    q_val = int(norm)
                    for s_tok, s_start, s_end in size_ranges:
                        if s_start <= q_col < s_end:
                            mapping[s_tok] += q_val
                            break

        mappings.append(mapping)

    return mappings

# Loop through PDFs and extract tables
all_results = {}
sizes_list = []
quantities_list = []

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        pages_text = extract_pdf_text_preserve_layout(pdf_path)

        file_results = []
        for page_num, text in pages_text.items():
            mappings = extract_size_qty_mappings(text)
            if mappings:
                file_results.extend(mappings)

        if file_results:
            all_results[filename] = file_results

            # Flatten as they appear (preserve order inside each block)
            for mapping in file_results:
                for size, qty in mapping.items():
                    sizes_list.append(size)
                    quantities_list.append(str(qty))

# --- Print Summary ---
for fname, blocks in all_results.items():
    print(f"\n=== Results from {fname} ===")
    for i, mapping in enumerate(blocks, 1):
        print(f"Block {i}: {mapping}")

print("\n--- Flattened Lists ---")
print("sizes_list =", sizes_list)
print("LENGTHHHHHHHH",len(sizes_list))
print("quantities_list =", quantities_list)



=== Results from PO_CGR-05B40767 Longsleeve, T-Shirts_241219 - Copy.pdf ===
Block 1: {'XS': 400, 'S': 1320, 'M': 1220, 'L': 840, 'XL': 1280, 'XXL': 240, '3XL': 50, '4XL': 250, '5XL': 20}
Block 2: {'XS': 0, 'S': 250, 'M': 850, 'L': 90, 'XL': 430, 'XXL': 90, '3XL': 20, '4XL': 10}
Block 3: {'XXS': 600, 'XS': 4730, 'S': 14472, 'M': 22882, 'L': 12690, 'XL': 11410, 'XXL': 4100, '3XL': 800, '4XL': 1050, '5XL': 300, '6XL': 20, '7XL': 100}
Block 4: {'XXS': 0, 'XS': 1000, 'S': 4000, 'M': 8500, 'L': 400, 'XL': 5000, 'XXL': 2200, '3XL': 200, '4XL': 400, '5XL': 150, '6XL': 0, '7XL': 0}
Block 5: {'XXS': 0, 'XS': 7320, 'S': 15762, 'M': 25032, 'L': 15520, 'XL': 8350, 'XXL': 1100, '3XL': 1500, '4XL': 1450, '5XL': 150, '6XL': 0, '7XL': 120}
Block 6: {'XXS': 0, 'XS': 200, 'S': 252, 'M': 3002, 'L': 3500, 'XL': 300, 'XXL': 400, '3XL': 100, '4XL': 0, '5XL': 30, '6XL': 0, '7XL': 0}

--- Flattened Lists ---
sizes_list = ['XS', 'S', 'M', 'L', 'XL', 'XXL', '3XL', '4XL', '5XL', 'XS', 'S', 'M', 'L', 'XL', 'XXL',